In [1]:
!pip install -U sentence-transformers rank_bm25

In [2]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")


#We use the Bi-Encoder to encode all passages, so that we can use it with semantic search
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 512     
top_k = 32                          

#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')



/home/elson/factcheck/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import string

def remove_punctuation(text):
    if not isinstance(text, str):
        text = str(text)  # Convert non-strings to strings
    return text.translate(str.maketrans('', '', string.punctuation))

In [4]:
# Import Module 
import os 

# Folder Path 
path_to_zip_file = "/home/elson/chunks.zip"
directory_to_extract_to = "/home/elson/"
import zipfile
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

def read_text_file(file_path): 
    with open(file_path, 'r', encoding="utf8") as f:
        doc = f.read()
    return doc
        

In [5]:
import os
import nltk
from nltk.tokenize import sent_tokenize

# Download necessary NLTK models
nltk.download('punkt')

# Function to read text from a file
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Path to the corpus directory
path = "/home/elson/content/chunks/"
evidence_documents = []

# Iterate through each text file in the directory
for efile in os.listdir(path):
    if efile.endswith(".txt"):
        print(efile)  # Print the file name
        file_path = os.path.join(path, efile)  # Construct full file path
        evidence = read_text_file(file_path)  # Read text from the file
        evidence_documents.append(evidence)


[nltk_data] Downloading package punkt to /home/elson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


chunk3451.txt
chunk13700.txt
chunk16525.txt
chunk14262.txt
chunk8119.txt
chunk4744.txt
chunk4775.txt
chunk10242.txt
chunk13994.txt
chunk1943.txt
chunk17600.txt
chunk3520.txt
chunk5734.txt
chunk12903.txt
chunk9955.txt
chunk18752.txt
chunk527.txt
chunk12697.txt
chunk16120.txt
chunk12106.txt
chunk8461.txt
chunk17387.txt
chunk1397.txt
chunk6500.txt
chunk7109.txt
chunk8526.txt
chunk568.txt
chunk17782.txt
chunk8028.txt
chunk1332.txt
chunk15905.txt
chunk13382.txt
chunk19136.txt
chunk2818.txt
chunk19162.txt
chunk12135.txt
chunk2733.txt
chunk121.txt
chunk17111.txt
chunk16435.txt
chunk17548.txt
chunk11821.txt
chunk14539.txt
chunk8608.txt
chunk9926.txt
chunk14379.txt
chunk10416.txt
chunk14261.txt
chunk5428.txt
chunk3280.txt
chunk576.txt
chunk466.txt
chunk3566.txt
chunk16964.txt
chunk3821.txt
chunk8216.txt
chunk3827.txt
chunk7080.txt
chunk4595.txt
chunk15070.txt
chunk6616.txt
chunk18400.txt
chunk12576.txt
chunk7205.txt
chunk7880.txt
chunk3707.txt
chunk16997.txt
chunk11355.txt
chunk12375.txt
chunk1

chunk8869.txt
chunk5695.txt
chunk12527.txt
chunk2930.txt
chunk3584.txt
chunk7697.txt
chunk5264.txt
chunk14324.txt
chunk1495.txt
chunk7274.txt
chunk9943.txt
chunk12514.txt
chunk5654.txt
chunk6594.txt
chunk4253.txt
chunk11566.txt
chunk774.txt
chunk283.txt
chunk3694.txt
chunk9015.txt
chunk5746.txt
chunk2307.txt
chunk5429.txt
chunk4728.txt
chunk6384.txt
chunk9681.txt
chunk8435.txt
chunk14151.txt
chunk11457.txt
chunk428.txt
chunk461.txt
chunk1068.txt
chunk11067.txt
chunk11527.txt
chunk13980.txt
chunk2617.txt
chunk4749.txt
chunk13235.txt
chunk11570.txt
chunk517.txt
chunk5750.txt
chunk9309.txt
chunk5735.txt
chunk18496.txt
chunk619.txt
chunk4198.txt
chunk10916.txt
chunk10097.txt
chunk9168.txt
chunk588.txt
chunk19703.txt
chunk4301.txt
chunk11290.txt
chunk11366.txt
chunk7717.txt
chunk14709.txt
chunk1321.txt
chunk11777.txt
chunk2962.txt
chunk12379.txt
chunk14277.txt
chunk12757.txt
chunk3336.txt
chunk19005.txt
chunk11625.txt
chunk3120.txt
chunk2184.txt
chunk6682.txt
chunk8599.txt
chunk9050.txt
chu

chunk5491.txt
chunk3281.txt
chunk11093.txt
chunk11718.txt
chunk15777.txt
chunk18706.txt
chunk7670.txt
chunk11548.txt
chunk941.txt
chunk9459.txt
chunk10490.txt
chunk9263.txt
chunk8107.txt
chunk18176.txt
chunk8551.txt
chunk16309.txt
chunk7284.txt
chunk2030.txt
chunk734.txt
chunk1110.txt
chunk642.txt
chunk10893.txt
chunk18217.txt
chunk3534.txt
chunk13004.txt
chunk8295.txt
chunk11961.txt
chunk10153.txt
chunk14194.txt
chunk12459.txt
chunk19129.txt
chunk17660.txt
chunk2564.txt
chunk1712.txt
chunk11893.txt
chunk19124.txt
chunk2891.txt
chunk6365.txt
chunk18657.txt
chunk11303.txt
chunk749.txt
chunk17425.txt
chunk6200.txt
chunk4323.txt
chunk6916.txt
chunk9392.txt
chunk12609.txt
chunk17384.txt
chunk3173.txt
chunk6997.txt
chunk15090.txt
chunk14993.txt
chunk5132.txt
chunk83.txt
chunk2057.txt
chunk2243.txt
chunk6856.txt
chunk15402.txt
chunk13242.txt
chunk14924.txt
chunk724.txt
chunk11986.txt
chunk9095.txt
chunk7883.txt
chunk17370.txt
chunk19619.txt
chunk5389.txt
chunk7222.txt
chunk12500.txt
chunk569

chunk4569.txt
chunk10142.txt
chunk8679.txt
chunk871.txt
chunk490.txt
chunk11324.txt
chunk19529.txt
chunk15403.txt
chunk10555.txt
chunk19081.txt
chunk11049.txt
chunk1802.txt
chunk14728.txt
chunk16852.txt
chunk19388.txt
chunk12655.txt
chunk8857.txt
chunk15388.txt
chunk13965.txt
chunk12336.txt
chunk9199.txt
chunk11029.txt
chunk14587.txt
chunk12307.txt
chunk15586.txt
chunk5463.txt
chunk869.txt
chunk1638.txt
chunk6908.txt
chunk12269.txt
chunk11454.txt
chunk14217.txt
chunk11825.txt
chunk2053.txt
chunk1841.txt
chunk10673.txt
chunk16035.txt
chunk18875.txt
chunk9356.txt
chunk9189.txt
chunk563.txt
chunk18462.txt
chunk15947.txt
chunk12644.txt
chunk5058.txt
chunk616.txt
chunk2396.txt
chunk18606.txt
chunk14948.txt
chunk9008.txt
chunk6966.txt
chunk5961.txt
chunk2586.txt
chunk9002.txt
chunk16184.txt
chunk9204.txt
chunk15873.txt
chunk13494.txt
chunk2538.txt
chunk4026.txt
chunk13387.txt
chunk18758.txt
chunk15630.txt
chunk15259.txt
chunk9401.txt
chunk8561.txt
chunk6800.txt
chunk1632.txt
chunk5170.txt
ch

...

chunk2006.txt
chunk10256.txt
chunk11602.txt
chunk14097.txt
chunk9817.txt
chunk11403.txt
chunk6153.txt
chunk12516.txt
chunk18819.txt
chunk7011.txt
chunk18453.txt
chunk5982.txt
chunk15953.txt
chunk1101.txt
chunk9734.txt
chunk16742.txt
chunk3677.txt
chunk7357.txt
chunk6866.txt
chunk9505.txt
chunk16437.txt
chunk18915.txt
chunk7232.txt
chunk7453.txt
chunk5298.txt
chunk19427.txt
chunk14648.txt
chunk2903.txt
chunk12681.txt
chunk19297.txt
chunk16072.txt
chunk3648.txt
chunk16004.txt
chunk17768.txt
chunk11686.txt
chunk16221.txt
chunk12767.txt
chunk19478.txt
chunk18668.txt
chunk18266.txt
chunk15565.txt
chunk2449.txt
chunk12195.txt
chunk12831.txt
chunk7878.txt
chunk7965.txt
chunk13233.txt
chunk17837.txt
chunk13356.txt
chunk16094.txt
chunk9433.txt
chunk16504.txt
chunk3175.txt
chunk5266.txt
chunk12845.txt
chunk19674.txt
chunk157.txt
chunk10734.txt
chunk8817.txt
chunk649.txt
chunk4095.txt
chunk19531.txt
chunk7014.txt
chunk3549.txt
chunk504.txt
chunk13922.txt
chunk1580.txt
chunk9288.txt
chunk14273.txt

chunk6774.txt
chunk8532.txt
chunk16043.txt
chunk8319.txt
chunk5815.txt
chunk502.txt
chunk4537.txt
chunk17377.txt
chunk12885.txt
chunk10486.txt
chunk15014.txt
chunk17336.txt
chunk8890.txt
chunk158.txt
chunk17005.txt
chunk19436.txt
chunk17853.txt
chunk3995.txt
chunk108.txt
chunk4107.txt
chunk1644.txt
chunk17367.txt
chunk1420.txt
chunk5114.txt
chunk1886.txt
chunk12907.txt
chunk2275.txt
chunk914.txt
chunk19424.txt
chunk4777.txt
chunk97.txt
chunk9181.txt
chunk13531.txt
chunk9101.txt
chunk10405.txt
chunk15439.txt
chunk8560.txt
chunk19050.txt
chunk6937.txt
chunk8781.txt
chunk4307.txt
chunk6376.txt
chunk4297.txt
chunk17428.txt
chunk3656.txt
chunk2309.txt
chunk15889.txt
chunk14738.txt
chunk6546.txt
chunk16242.txt
chunk16971.txt
chunk18974.txt
chunk9231.txt
chunk4637.txt
chunk7488.txt
chunk14882.txt
chunk2103.txt
chunk8304.txt
chunk18769.txt
chunk1234.txt
chunk10569.txt
chunk11834.txt
chunk5029.txt
chunk5506.txt
chunk12762.txt
chunk18103.txt
chunk6122.txt
chunk4234.txt
chunk16393.txt
chunk143.tx

chunk627.txt
chunk5369.txt
chunk11261.txt
chunk15841.txt
chunk12417.txt
chunk4270.txt
chunk5402.txt
chunk15702.txt
chunk9687.txt
chunk1968.txt
chunk2292.txt
chunk13038.txt
chunk17498.txt
chunk14591.txt
chunk1678.txt
chunk7868.txt
chunk7457.txt
chunk4545.txt
chunk10062.txt
chunk18705.txt
chunk7831.txt
chunk10559.txt
chunk14475.txt
chunk819.txt
chunk9576.txt
chunk12163.txt
chunk4654.txt
chunk11004.txt
chunk19292.txt
chunk2471.txt
chunk16965.txt
chunk17915.txt
chunk13380.txt
chunk2901.txt
chunk17192.txt
chunk15153.txt
chunk9494.txt
chunk10683.txt
chunk18459.txt
chunk14884.txt
chunk10282.txt
chunk18527.txt
chunk17670.txt
chunk7840.txt
chunk5171.txt
chunk15479.txt
chunk3014.txt
chunk12136.txt
chunk17656.txt
chunk11164.txt
chunk17375.txt
chunk17981.txt
chunk15103.txt
chunk18910.txt
chunk12323.txt
chunk6956.txt
chunk5566.txt
chunk7317.txt
chunk16690.txt
chunk1987.txt
chunk2122.txt
chunk15868.txt
chunk12395.txt
chunk8260.txt
chunk5368.txt
chunk15816.txt
chunk11392.txt
chunk8842.txt
chunk12901.

chunk422.txt
chunk12340.txt
chunk13715.txt
chunk712.txt
chunk13167.txt
chunk776.txt
chunk7734.txt
chunk10796.txt
chunk17356.txt
chunk7999.txt
chunk3687.txt
chunk9222.txt
chunk19572.txt
chunk6299.txt
chunk17107.txt
chunk1873.txt
chunk1187.txt
chunk18984.txt
chunk18977.txt
chunk4184.txt
chunk12090.txt
chunk17025.txt
chunk18216.txt
chunk7452.txt
chunk17063.txt
chunk11726.txt
chunk4795.txt
chunk8264.txt
chunk16777.txt
chunk4119.txt
chunk16679.txt
chunk4842.txt
chunk12803.txt
chunk18068.txt
chunk4510.txt
chunk10977.txt
chunk10948.txt
chunk2861.txt
chunk18566.txt
chunk2431.txt
chunk14121.txt
chunk13045.txt
chunk18512.txt
chunk4718.txt
chunk13744.txt
chunk9019.txt
chunk19068.txt
chunk11476.txt
chunk2537.txt
chunk17222.txt
chunk2129.txt
chunk4045.txt
chunk3855.txt
chunk16733.txt
chunk8093.txt
chunk6813.txt
chunk17822.txt
chunk2784.txt
chunk11042.txt
chunk421.txt
chunk12245.txt
chunk19186.txt
chunk15552.txt
chunk2354.txt
chunk2320.txt
chunk12122.txt
chunk17901.txt
chunk8854.txt
chunk17326.txt
c

chunk7679.txt
chunk15672.txt
chunk12225.txt
chunk16038.txt
chunk17838.txt
chunk7458.txt
chunk1139.txt
chunk6703.txt
chunk12763.txt
chunk17961.txt
chunk10573.txt
chunk3874.txt
chunk16268.txt
chunk2207.txt
chunk432.txt
chunk5041.txt
chunk17106.txt
chunk9995.txt
chunk13337.txt
chunk13967.txt
chunk8010.txt
chunk18928.txt
chunk9412.txt
chunk13656.txt
chunk5950.txt
chunk18031.txt
chunk10531.txt
chunk11654.txt
chunk4266.txt
chunk9035.txt
chunk9810.txt
chunk11308.txt
chunk7749.txt
chunk13650.txt
chunk2947.txt
chunk14023.txt
chunk996.txt
chunk13059.txt
chunk5728.txt
chunk169.txt
chunk8921.txt
chunk15485.txt
chunk18071.txt
chunk16527.txt
chunk8766.txt
chunk1879.txt
chunk7930.txt
chunk17681.txt
chunk5173.txt
chunk4153.txt
chunk19598.txt
chunk9383.txt
chunk3369.txt
chunk5407.txt
chunk8136.txt
chunk8166.txt
chunk5330.txt
chunk9044.txt
chunk13090.txt
chunk16415.txt
chunk8105.txt
chunk1186.txt
chunk7351.txt
chunk16323.txt
chunk17701.txt
chunk1463.txt
chunk15145.txt
chunk4726.txt
chunk15182.txt
chunk1

In [6]:
evidence_documents[500]

'These cytotoxic compounds are responsible for the destruction of pigment cells, resulting in skin depigmentation.\nIn addition, hydroquinone acts on the rate limiting enzyme tyrosinase, thereby inhibiting the enzymatic oxidation of tyrosine to dopaquinone.\nIt is common for hydroquinone to have adverse effects, particularly among individuals with dark skin tones, including skin irritation and allergic contact dermatitis.\nRare instances of permanent depigmentation and exogenous ochronosis have been reported with long-term use.\nTherefore, hydroquinone has been banned for use as skin whiteners in skincare cosmetics in Europe by the Cosmetic Directive 2000/6/EC.\nHowever, in the United States (US), hydroquinone is available for non-cosmetic use, both as prescription and OTC products in concentrations ranging from 0.4% to 5.0%.\nSince its medical use is under the purview of US Food and Drug Administration (FDA), hydroquinone-containing OTC products are no longer generally recognized as s

In [7]:
len(evidence_documents)

19736

In [8]:
preprocessed_documents = [remove_punctuation(doc) for doc in evidence_documents]
preprocessed_documents = [doc.lower() for doc in preprocessed_documents]
preprocessed_documents = [doc.replace('\n', ' ') for doc in preprocessed_documents]
preprocessed_documents = [doc.replace('\t', ' ') for doc in preprocessed_documents]


In [9]:
device = 'cuda:3'

In [10]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Remove stopwords from each document in the list
preprocessed_documents = [' '.join([word for word in doc.split() if word not in stop_words]) 
                          for doc in preprocessed_documents]

[nltk_data] Downloading package stopwords to /home/elson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
corpus_embeddings = bi_encoder.encode(evidence_documents, convert_to_tensor=True, show_progress_bar=True).to(device)

Batches: 100%|██████████| 617/617 [00:18<00:00, 32.85it/s]


In [12]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np


# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc


tokenized_corpus = []
for doc in tqdm(evidence_documents):
    tokenized_corpus.append(bm25_tokenizer(doc))

bm25 = BM25Okapi(tokenized_corpus)

100%|██████████| 19736/19736 [00:02<00:00, 9283.30it/s]


In [13]:
# This function will search all wikipedia articles for passages that
# answer the query
def search(query):
    print("Input question:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -5)[-5:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
    
    print("Top-3 lexical search (BM25) hits")
    for hit in bm25_hits[0:10]:
        print("\t{:.3f}\t{}".format(hit['score'], evidence_documents[hit['corpus_id']].replace("\n", " ")))

    ##### Semantic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, evidence_documents[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from bi-encoder
    print("\n-------------------------\n")
    print("Top-3 Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits[0:10]:
        print("\t{:.3f}\t{}".format(hit['score'], evidence_documents[hit['corpus_id']].replace("\n", " ")))

    # Output of top-5 hits from re-ranker
    print("\n-------------------------\n")
    print("Top-3 Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    for hit in hits[0:10]:
        print("\t{:.3f}\t{}".format(hit['cross-score'], evidence_documents[hit['corpus_id']].replace("\n", " ")))

In [14]:
search("You can't get thyroid disease if you're young.")

Input question: You can't get thyroid disease if you're young.
Top-3 lexical search (BM25) hits
	15.253	Will I need to do anything to prepare for the test? You don't need any special preparations for a TSH blood test. If your health care provider has ordered other blood tests, you may need to. (not eat or drink) for several hours before the test. Your provider will let you know if there are any special instructions to follow. There is very little risk to having a blood test. You may have slight pain or bruising at the spot where the needle was put in, but most symptoms go away quickly. Your TSH test results can tell you if your thyroid is making too much or too little thyroid. But the test can't explain why your TSH levels may be too high or too low. If your test results aren't normal, your provider will probably order. to find out what's causing your thyroid problem. Graves' disease, the most common cause of hyperthyroidism. Hashimoto's disease, that the most common cause of hypothyro

In [24]:
def search_ce(query):
    # Initializations
    top_hits = [None]  # To store the top hit
    
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]
    
    cross_inp = [[query, evidence_documents[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    for idx, hit in enumerate(hits):
        hit['cross-score'] = cross_scores[idx]
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    
    for i, hit in enumerate(hits[:1]):
        if i < len(top_hits):
            top_hits[i] = evidence_documents[hit['corpus_id']]

    return top_hits

In [23]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [29]:
import pandas as pd
df = pd.read_excel("/home/elson/Claims_withGeminiAnnotation.xlsx", engine='openpyxl')

In [30]:
# Assuming df is your DataFrame
# Convert all claims to strings
df['claim'] = df['claim'].astype(str)

# Apply the function and expand the returned list into separate columns
df[['top_chunk']] = df['claim'].apply(lambda claim: pd.Series(search_ce(claim)))


In [31]:
df.head()

,folder,filename,claim,label,url,GOLD EXPLANATION,CATEGORY,gemini_label,gemini_explanation,top_chunk
0,/content/drive/MyDrive/images/myths on urticaria,mythsonurticaria1.jpeg,Eating chocolate will cause acne.,SUPPORTED,https://www.jaad.org/article/S0190-9622(16)013...,The chocolate consumption group had a statisti...,Skin,REFUTED,There is no scientific evidence to support the...,Chocolate consumption modulates cytokine produ...
1,/content/drive/MyDrive/images/myths on urticaria,mythsonurticaria1.jpeg,You can get a cold from being in the rain.,SUPPORTED,https://pubmed.ncbi.nlm.nih.gov/17705968/,Exposure to cold has often been associated wi...,General Health,REFUTED,"The common cold is caused by viruses, not by b...",Exposure to cold has often been associated wi...
2,/content/drive/MyDrive/images/myths on urticaria,mythsonurticaria1.jpeg,Stress can cause acne.,SUPPORTED,https://medicaljournalssweden.se/actadv/articl...,"Based on this study, increased stress does not...",Skin,SUPPORTED,There is evidence to suggest that stress can t...,Both active acne and post-inflammatory hyperpi...
3,/content/drive/MyDrive/images/myths on urticaria,mythsonurticaria1.jpeg,You can prevent acne by washing your face more...,NOT ENOUGH INFORMATION,https://www.tandfonline.com/doi/full/10.1080/0...,Washing and over-the-counter cleansers are com...,Skin,REFUTED,Washing your face more often does not prevent ...,Systemic therapies are usually prescribed for ...
4,/content/drive/MyDrive/images/myths on vascula...,mythsonvascularsurgery3.jpeg,Varicose veins are caused by standing too much.,SUPPORTED,https://www.sjweh.fi/article/562,"For men working mostly in a standing position,...",Vascular,NOT ENOUGH INFORMATION,While standing for long periods of time can co...,"JAQUELINE RAETZ, MD, MEGAN WILSON, MD, AND KIM..."
